In [33]:
from typing import Union
from datetime import datetime, date, time
from dateutil import parser
import earthaccess
import geopandas as gpd
from shapely.geometry.polygon import orient
from sentinel_tiles import sentinel_tiles
from emit_tools import CMR_GRANULE_SEARCH_URL
from emit_tools import generate_CMR_daterange_string

In [40]:
# earthaccess.login(strategy="netrc")
earthaccess.login(persist=True)

In [3]:
earthaccess.search_data?

Signature: earthaccess.search_data(count: int = -1, **kwargs: Any) -> List[earthaccess.results.DataGranule]
Docstring:
Search dataset granules using NASA's CMR.

[https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)

Parameters:
    count: Number of records to get, -1 = all
    kwargs (Dict):
        arguments to CMR:

        * **short_name**: dataset short name, e.g. ATL08
        * **version**: dataset version
        * **doi**: DOI for a dataset
        * **daac**: e.g. NSIDC or PODAAC
        * **provider**: particular to each DAAC, e.g. POCLOUD, LPDAAC etc.
        * **temporal**: a tuple representing temporal bounds in the form
          `(date_from, date_to)`
        * **bounding_box**: a tuple representing spatial bounds in the form
          `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`

Returns:
    a list of DataGranules that can be used to access the granule files by using
 

In [41]:
earthaccess.search_data(
    short_name="EMITL2ARFL",
    temporal=("2022-09-03", "2022-09-04"),
    point=(-62.1123,-39.89402),
    count=3
)

[Collection: {'ShortName': 'EMITL2ARFL', 'Version': '001'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -62.08873748779297, 'Latitude': -39.2420539855957}, {'Longitude': -62.51200866699219, 'Latitude': -39.943302154541016}, {'Longitude': -61.660133361816406, 'Latitude': -40.45749282836914}, {'Longitude': -61.23686218261719, 'Latitude': -39.75624465942383}, {'Longitude': -62.08873748779297, 'Latitude': -39.2420539855957}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2022-09-03T16:31:29Z', 'EndingDateTime': '2022-09-03T16:31:41Z'}}
 Size(MB): 3578.7448024749756
 Data: ['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc', 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFLUNCERT_001_20220903T163129_

In [9]:
tile_polygon = sentinel_tiles.footprint("11SPS")
tile_polygon

In [27]:
list(tile_polygon.centroid.geometry.coords)[0]

(-115.34318572284748, 32.932906910398486)

In [31]:
earthaccess.search_data(
    short_name="EMITL2ARFL",
    temporal=("2024-05-01", "2024-06-01"),
    point=list(tile_polygon.centroid.geometry.coords)[0],
    count=3
)

IndexError: list index out of range

In [10]:
tile_polygon.wkt

'POLYGON ((-115.92423373599996 33.43490918800006, -114.74363157999994 33.419046707000064, -114.76862412 32.429359161000036, -115.93615876899997 32.444635394000045, -115.92423373599996 33.43490918800006))'

In [14]:
polygon = list(orient(tile_polygon, sign=1).exterior.coords)
print(type(polygon))
polygon

<class 'list'>


[(-115.92423373599996, 33.43490918800006),
 (-115.93615876899997, 32.444635394000045),
 (-114.76862412, 32.429359161000036),
 (-114.74363157999994, 33.419046707000064),
 (-115.92423373599996, 33.43490918800006)]

In [42]:
geojson = gpd.read_file('isla_gaviota.geojson')
geojson.geometry

0    POLYGON ((-62.14758 -39.88951, -62.169 -39.876...
Name: geometry, dtype: geometry

In [43]:
polygon = list(geojson.geometry[0].exterior.coords)
polygon

[(-62.147583513919045, -39.88950549416461),
 (-62.16899895047814, -39.87693893067732),
 (-62.19419358172446, -39.90641838472922),
 (-62.20427143422259, -39.94071456822524),
 (-62.1318368693898, -39.95230375907932),
 (-62.11609022486114, -39.92091182572591),
 (-62.125538211578245, -39.895787912197314),
 (-62.147583513919045, -39.88950549416461)]

In [44]:
earthaccess.search_data(
    short_name="EMITL2ARFL",
    temporal=("2024-05-01", "2024-06-01"),
    polygon=polygon,
    count=3
)

IndexError: list index out of range

In [16]:
tile_bbox = sentinel_tiles.footprint("11SPS").bbox
tile_bbox

BBox(xmin=-115.93615876899997, ymin=32.429359161000036, xmax=-114.74363157999994, ymax=33.43490918800006, crs="EPSG:4326")

In [17]:
tuple(tile_bbox)

(-115.93615876899997,
 32.429359161000036,
 -114.74363157999994,
 33.43490918800006)

In [18]:
earthaccess.search_data(
    short_name="EMITL2ARFL",
    temporal=("2024-01-01", "2024-02-01"),
    bounding_box=tuple(tile_bbox),
    count=3
)

IndexError: list index out of range

In [ ]:
orient(tile_polygon, sign=1.0)

In [ ]:
list(orient(tile_polygon, sign=1.0).exterior.coords)

In [19]:
results = earthaccess.search_data(
    short_name='EMITL2ARFL',
    polygon=list(orient(tile_polygon, sign=1.0).exterior.coords),
    temporal=('2022-09-03','2022-09-04'),
    count=100
)

results

IndexError: list index out of range